In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error,accuracy_score

In [2]:
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('train.csv')
train=train.drop(columns=["Submitted_for_Approval","Prod_Category_A","Last_Activity","Actual_Delivery_Date"])

In [3]:
#train_Product_Family=train.groupby(["Product_Family","Stage"]).agg({"Stage":"count"}).rename(columns={"Stage":"Stage_count"}).reset_index().sort_values("Stage_count",ascending=False)
#train_Product_Name=train.groupby(["Product_Name","Stage"]).agg({"Stage":"count"}).rename(columns={"Stage":"Stage_count"}).reset_index().sort_values("Stage_count",ascending=False)

In [4]:
#train_Product_Family_100=train_Product_Family.loc[train_Product_Family.Stage_count>100]
#train_Product_Family_100.groupby(["Product_Family","Stage"]).agg({"Stage_count":"sum"})

In [5]:
#train.Opportunity_ID.value_counts().head(5)

In [7]:
#train.loc[train.Opportunity_ID==12460][["Product_Family","Opportunity_Name","Product_Family","Product_Name"]]

In [6]:
train=train.loc[(train.Stage=="Closed Lost") | (train.Stage=="Closed Won")]
train.Stage=train.Stage.replace(['Closed Won','Closed Lost'],[1,0])
stage=train.Stage
train_1=train.iloc[:,:8]

In [8]:
y = pd.get_dummies(train_1.Bureaucratic_Code, prefix='OHE')

In [9]:
train_1["OHE_Bureaucratic_Code_0"]=y["OHE_Bureaucratic_Code_0"]
train_1["OHE_Bureaucratic_Code_1"]=y["OHE_Bureaucratic_Code_1"]
train_1["OHE_Bureaucratic_Code_2"]=y["OHE_Bureaucratic_Code_2"]
train_1["OHE_Bureaucratic_Code_3"]=y["OHE_Bureaucratic_Code_3"]
train_1["OHE_Bureaucratic_Code_4"]=y["OHE_Bureaucratic_Code_4"]
train_1["OHE_Bureaucratic_Code_5"]=y["OHE_Bureaucratic_Code_5"]
train_1["OHE_Bureaucratic_Code_6"]=y["OHE_Bureaucratic_Code_6"]

In [10]:
train_1=train_1.drop(columns=["Region","Territory","Bureaucratic_Code","ID"])

In [11]:
train_1.shape

(16883, 11)

In [12]:
X_train, X_test, y_train, y_test = \
    train_test_split(train_1, stage, test_size=0.2, random_state=123)

In [13]:
X_train.shape,X_test.shape

((13506, 11), (3377, 11))

In [14]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [15]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [16]:
preds = xg_reg.predict(X_test)

In [17]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.480286


In [18]:
comp=pd.DataFrame({'real':y_test,'pred':preds})

In [19]:
preds

array([0.5068309 , 0.5068309 , 0.60698503, ..., 0.42321703, 0.5068309 ,
       0.60698503], dtype=float32)

In [20]:
comp[comp.pred>0.5]=1
comp[comp.pred<=0.5]=0

In [21]:
ac=accuracy_score(comp.real,comp.pred)

In [22]:
ac

1.0

Pruebas En El Test

In [23]:
test.shape

(2551, 51)

In [25]:
test=test.drop(columns=["Submitted_for_Approval","Prod_Category_A","Last_Activity","Actual_Delivery_Date"])

In [26]:
test=test.sort_values("ID",ascending=True)

In [27]:
test.to_csv('test_order.csv',index=False)

In [28]:
test_1=test.iloc[:,:8]

In [29]:
test_1.head(2)

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code
1562,4870,Americas,NW America,1,1,0,0,Bureaucratic_Code_4
1563,4875,Americas,NW America,1,1,0,0,Bureaucratic_Code_4


In [30]:
y_ = pd.get_dummies(test_1.Bureaucratic_Code, prefix='OHE')

In [31]:
test_1["OHE_Bureaucratic_Code_0"]=y_["OHE_Bureaucratic_Code_0"]
test_1["OHE_Bureaucratic_Code_1"]=y_["OHE_Bureaucratic_Code_1"]
test_1["OHE_Bureaucratic_Code_2"]=y_["OHE_Bureaucratic_Code_2"]
test_1["OHE_Bureaucratic_Code_3"]=0#y_["OHE_Bureaucratic_Code_3"]
test_1["OHE_Bureaucratic_Code_4"]=y_["OHE_Bureaucratic_Code_4"]
test_1["OHE_Bureaucratic_Code_5"]=y_["OHE_Bureaucratic_Code_5"]
test_1["OHE_Bureaucratic_Code_6"]=0#y_["OHE_Bureaucratic_Code_6"]

In [32]:
#y = pd.get_dummies(train_1.Bureaucratic_Code, prefix='OHE')
test_1=test_1.drop(columns=["Region","Territory","Bureaucratic_Code","ID"])

In [33]:
test_1.head(2)

,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,OHE_Bureaucratic_Code_0,OHE_Bureaucratic_Code_1,OHE_Bureaucratic_Code_2,OHE_Bureaucratic_Code_3,OHE_Bureaucratic_Code_4,OHE_Bureaucratic_Code_5,OHE_Bureaucratic_Code_6
1562,1,1,0,0,0,0,0,0,1,0,0
1563,1,1,0,0,0,0,0,0,1,0,0


In [34]:
preds = xg_reg.predict(test_1)

In [35]:
preds.shape,test_1.shape

((2551,), (2551, 11))

In [36]:
comp=pd.DataFrame({'Target':preds})

In [46]:
test_pre=pd.DataFrame({'Opportunity_ID':test.Opportunity_ID})

In [47]:
test_pre['Target']=comp.Target

In [48]:
test_pre.head(2)

,Opportunity_ID,Target
1562,11780,0.423217
1563,11780,0.423217


In [49]:
test_pre=test_pre.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()

In [54]:
#test_pre.loc[test_pre.Target>=0.5].Target=1
test_pre["Target"] = np.where(test_pre["Target"]>0.5, 1 ,0)

In [55]:
test_pre

,Opportunity_ID,Target
0,10689,1
1,10690,0
2,10691,1
3,10692,1
4,10693,1
5,10694,1
6,10695,0
7,10696,0
8,10697,1
9,10698,1


In [56]:
#test_pre[test_pre.Target>=0.5]=1
#test_pre[test_pre.Target<0.5]=0

In [57]:
test_pre.shape

(1567, 2)

In [59]:
test_pre.to_csv('test_pre_.csv',index=False)